In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18216282
paper_name = 'schluter_conibear_2008' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [38]:
original_data = pd.read_excel('raw_data/DataSet1.xlsx', sheet_name='Table S1', skiprows=2)

In [39]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4814 x 9


In [40]:
original_data.head()

,ORF,Gene name,MATa 1,MATa 2,MATa 1.1,MATa 2.1,diploid 1,diploid 2,sorting index
0,YBL017C,PEP1,1.0,1.000000,1.0,1.000000,0.958264,0.993273,1.0
1,YJL053W,PEP8,1.0,1.000000,1.0,0.912331,0.971011,1.000000,1.0
2,YOR132W,VPS17,1.0,1.000000,1.0,1.000000,0.957967,0.893909,1.0
3,YHR012W,VPS29,1.0,1.000000,1.0,0.468719,1.000000,1.000000,1.0
4,YPL120W,VPS30,1.0,0.998057,1.0,1.000000,1.000000,0.977717,1.0


In [41]:
original_data['orf'] = original_data['ORF'].astype(str)

In [42]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [43]:
original_data.loc[original_data['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [44]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [45]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Gene name, MATa 1, MATa 2, MATa 1.1, MATa 2.1, diploid 1, diploid 2, sorting index, orf]
Index: []


In [46]:
original_data.set_index('orf', inplace=True)

In [47]:
original_data = original_data[['MATa 1','MATa 2','MATa 1.1','MATa 2.1','diploid 1','diploid 2']].copy()

In [48]:
original_data = -original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [49]:
original_data = original_data.groupby(original_data.index).mean()

In [50]:
original_data.shape

(4785, 6)

In [51]:
original_data.columns = [16030, 16030, 16031, 16031, 16032, 16032]

In [52]:
original_data = original_data.T
original_data = original_data.groupby(original_data.index).mean()
original_data = original_data.T

In [53]:
original_data.shape

(4785, 3)

# Prepare the final dataset

In [54]:
data = original_data.copy()

In [55]:
dataset_ids = original_data.columns.values
datasets = datasets.reindex(index=dataset_ids)

In [56]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [57]:
data.head()

dataset_id,16030,16031,16032
data_type,value,value,value
orf,,,
YAL002W,-0.734342,-1.000000,-0.991316
YAL004W,-0.032270,-0.012043,-0.010930
YAL005C,-0.003323,-0.020758,-0.028784
YAL007C,-0.011382,-0.006527,-0.004714
YAL008W,-0.000837,-0.007604,-0.000157


## Subset to the genes currently in SGD

In [58]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 32


In [59]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16030,16031,16032
,data_type,value,value,value
gene_id,orf,,,
2,YAL002W,-0.734342,-1.000000,-0.991316
1863,YAL004W,-0.032270,-0.012043,-0.010930
4,YAL005C,-0.003323,-0.020758,-0.028784
5,YAL007C,-0.011382,-0.006527,-0.004714
6,YAL008W,-0.000837,-0.007604,-0.000157


# Normalize

In [60]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [61]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [62]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16030,16031,16032,16030,16031,16032
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
2,YAL002W,-0.734342,-1.000000,-0.991316,-6.199245,-10.745487,-8.346514
1863,YAL004W,-0.032270,-0.012043,-0.010930,-0.189760,-0.020971,-0.007050
4,YAL005C,-0.003323,-0.020758,-0.028784,0.058014,-0.115580,-0.158923
5,YAL007C,-0.011382,-0.006527,-0.004714,-0.010968,0.038901,0.045820
6,YAL008W,-0.000837,-0.007604,-0.000157,0.079297,0.027217,0.084591


# Print out

In [63]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [64]:
from IO.save_data_to_db3 import *

In [65]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/3 [00:00<?, ?it/s]

Deleting all datasets for PMID 18216282...
Inserting the new data...


100%|██████████| 3/3 [00:22<00:00,  7.58s/it]

Updating the data_modified_on field...
